In [1]:
import os
import clr
import sys
from imp import reload
from System import Array
from time import sleep, time
import numpy as np
from matplotlib import pyplot as plt
# sys.path.append(os.getcwd())
%matplotlib inline

import qcodes as qc
qc.show_subprocess_widget()

<IPython.core.display.Javascript object>

In [ ]:
import qcodes.instrument_drivers.spincore.PulseBlasterESRPRO as PulseBlaster_driver
import qcodes.instrument_drivers.lecroy.ArbStudio1104 as ArbStudio_driver
import qcodes.instrument_drivers.stanford_research.SIM900 as SIM900_driver
import qcodes.instrument_drivers.AlazarTech.ATS9440 as ATS_driver
import qcodes.instrument_drivers.AlazarTech.ATS_acquisition_controllers as ATS_control

for instrument_name in ['PulseBlaster', 'ArbStudio', 'SIM900', 'ATS', 'ATS_controller']:
    try:
        eval('{}.close()'.format(instrument_name))
    except:
        pass
    eval('reload({}_driver)'.format(instrument_name))

PulseBlaster = PulseBlasterESRPRO.PulseBlaster('PulseBlaster')

dll_path = os.path.join(os.getcwd(),'lecroy_driver\\Library\\ArbStudioSDK.dll')
ArbStudio = ArbStudio1104.ArbStudio1104('ArbStudio', dll_path)

SIM900 = SIM900_driver.SIM900('SIM900', 'GPIB0::4::INSTR')
TG, LB, RB, TGAC, SRC, _, DS, DF = [eval('SIM900.chan{}'.format(i)) for i in range(1,9)]

ATS = ATS_driver.ATS9440('ATS', server_name='Alazar_server')
ATS_controller = ATS_control.Average_AcquisitionController(name='ATS_control', 
                                                           alazar_name='ATS',
                                                           server_name='Alazar_server')

ATS.config(trigger_source1='CHANNEL_C',
           trigger_level1=135,
           channel_range=2,
           sample_rate=1e6,
           coupling='DC')
ATS_controller.set_acquisitionkwargs(buffer_timeout=5000,
                                     samples_per_record=50000,
                                     records_per_buffer=1,
                                     buffers_per_acquisition=1,
                                     channel_selection='A')

In [5]:
for i in range(1,9):
    print('channel {}: {} V'.format(i, eval('SIM900.chan{}()'.format(i))))

channel 1: 15.54 V
channel 2: 3.8 V
channel 3: 3.8 V
channel 4: 1.675 V
channel 5: 0.25 V
channel 6: 0.0 V
channel 7: 2.646 V
channel 8: 2.646 V


In [103]:
ATS_controller.average_mode('point')
result = ATS_controller.acquisition()

plt.figure()
plt.plot(result[0])
plt.plot(result[1])

RuntimeError: *** error on Alazar_server ***
while executing query: ('ASK', 'cmd', (1, 'get', 'acquisition'), {})

(<bound method InstrumentServer.handle_cmd of <qcodes.instrument.server.InstrumentServer object at 0x0000000004803BE0>>, (1, 'get', 'acquisition'), {})
Traceback (most recent call last):
  File "c:\users\lab.blug12meas\documents\qcodes\qcodes\process\server.py", line 347, in _process_ask
    response = func(*args, **kwargs)
  File "c:\users\lab.blug12meas\documents\qcodes\qcodes\instrument\server.py", line 182, in handle_cmd
    return func(*args, **kwargs)
  File "c:\users\lab.blug12meas\documents\qcodes\qcodes\instrument\base.py", line 471, in get
    return self.parameters[param_name].get()
  File "c:\users\lab.blug12meas\documents\qcodes\qcodes\instrument\parameter.py", line 556, in get
    raise e
  File "c:\users\lab.blug12meas\documents\qcodes\qcodes\instrument\parameter.py", line 550, in get
    value = self._get()
  File "c:\users\lab.blug12meas\documents\qcodes\q

In [75]:
TGAC()

1.687

In [98]:
loc_provider = qc.data.location.FormatLocation(fmt='data/{date}/#{counter}_{name}_{time}')
qc.data.data_set.DataSet.location_provider=loc_provider

data = qc.Loop(TGAC.sweep(1.62,1.625,0.001), delay=0.003).each(
    ATS_controller.acquisition).run(name='testsweep')
plotQ = qc.QtPlot()
plotQ.add(data.ATS_control_acquisition)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = 'data/2016-08-19/#012_testsweep_15-22-59'
   <Type>   | <array_id>              | <array.name> | <array.shape>
   Setpoint | SIM900_chan4_set        | chan4        | (6,)
   Measured | ATS_control_acquisition | acquisition  | (6,)
started at 2016-08-19 15:23:00


In [97]:
ATS_controller.acquisition()

0.13714296875000009

In [81]:
import qcodes.instrument.parameter as parameter
import numpy as np

# Finally show that this instrument also works within a loop
dummy = parameter.ManualParameter(name="dummy")
dummy_detect = parameter.StandardParameter(name='dummy_detect',get_cmd=lambda: [np.random.rand(1)])

data = qc.Loop(dummy[0:50:1]).each(dummy_detect).run(name='AlazarTest',use_threads=False, background=False)
# qc.MatPlot(data.acquisition_controller_acquisition)

plotQ = qc.QtPlot()
plotQ.add(data.dummy_detect)
data.dummy_detect

In [200]:
SIM900.close()